In [2]:
// Add dependencies for Jupyter notebook
@file:Repository("https://repo1.maven.org/maven2/")
@file:DependsOn("org.jetbrains.kotlinx:kotlin-jupyter-api:0.17.0-755")

// Import the notebook API for HTML display
import org.jetbrains.kotlinx.jupyter.api.*
import java.io.BufferedReader
import java.io.File

data class Graph(
    private val adjacencyList: MutableMap<String, MutableList<String>> = mutableMapOf()
) {
    fun addEdge(from: String, to: String) {
        adjacencyList.getOrPut(from) { mutableListOf() }.add(to)
    }

    fun getEdges(node: String): List<String> = adjacencyList[node] ?: emptyList()
    
    fun getNodes(): Set<String> = adjacencyList.keys

    /**
     * Finds all paths from start node to end node using DFS
     * @param start The starting node
     * @param end The target node
     * @return List of all possible paths, where each path is a list of nodes
     */
    fun findAllPaths(start: String, end: String): List<List<String>> {
        val allPaths = mutableListOf<List<String>>()
        val currentPath = mutableListOf<String>()
        val visited = mutableSetOf<String>()
        
        dfsAllPaths(start, end, currentPath, visited, allPaths)
        return allPaths
    }
    
    private fun dfsAllPaths(
        current: String,
        target: String,
        currentPath: MutableList<String>,
        visited: MutableSet<String>,
        allPaths: MutableList<List<String>>
    ) {
        // Add current node to path and mark as visited
        currentPath.add(current)
        visited.add(current)
        
        if (current == target) {
            // Found a path to target, add copy to results
            allPaths.add(currentPath.toList())
        } else {
            // Continue exploring neighbors
            for (neighbor in getEdges(current)) {
                if (neighbor !in visited) {
                    dfsAllPaths(neighbor, target, currentPath, visited, allPaths)
                }
            }
        }
        
        // Backtrack: remove current node from path and visited set
        currentPath.removeAt(currentPath.size - 1)
        visited.remove(current)
    }

    fun toDot(graphName: String = "G"): String {
        return buildString {
            appendLine("digraph $graphName {")
            appendLine("    rankdir=LR;")
            appendLine("    node [shape=box, style=rounded];")
            appendLine()

            adjacencyList.forEach { (from, toList) ->
                toList.forEach { to ->
                    appendLine("    \"$from\" -> \"$to\";")
                }
            }

            appendLine("}")
        }
    }
    
    fun toHtmlTable(): String {
        return buildString {
            appendLine("""<div style="border: 1px solid #ccc; padding: 10px; margin: 10px;">""")
            appendLine("<h3>Graph Structure</h3>")
            appendLine("<table style='border-collapse: collapse; width: 100%;'>")
            appendLine("<tr><th style='border: 1px solid #ddd; padding: 8px;'>Node</th><th style='border: 1px solid #ddd; padding: 8px;'>Edges</th></tr>")
            
            adjacencyList.forEach { (node, edges) ->
                appendLine("<tr>")
                appendLine("<td style='border: 1px solid #ddd; padding: 8px;'>$node</td>")
                appendLine("<td style='border: 1px solid #ddd; padding: 8px;'>${edges.joinToString(", ")}</td>")
                appendLine("</tr>")
            }
            
            appendLine("</table>")
            appendLine("</div>")
        }
    }

    fun toSvg(): String {
        val dotContent = toDot()

        // Create temporary file for DOT content
        val dotFile = File.createTempFile("graph", ".dot")
        dotFile.writeText(dotContent)

        try {
            // Execute dot command to generate SVG
            val process = ProcessBuilder("dot", "-Tsvg", dotFile.absolutePath)
                .redirectErrorStream(true)
                .start()

            val svg = process.inputStream.bufferedReader().use(BufferedReader::readText)
            val exitCode = process.waitFor()

            if (exitCode != 0) {
                throw RuntimeException("GraphViz dot command failed with exit code $exitCode\n$svg")
            }

            return svg
        } finally {
            dotFile.delete()
        }
    }

    override fun toString(): String =
        adjacencyList.entries.joinToString("\n") { (node, edges) ->
            "$node -> ${edges.joinToString(", ")}"
        }
}

fun parseGraphFromLines(lines: Sequence<String>): Graph {
    val graph = Graph()

    lines.forEach { line ->
        val trimmed = line.trim()
        if (trimmed.isEmpty()) return@forEach

        val parts = trimmed.split(":")
        if (parts.size != 2) return@forEach

        val node = parts[0].trim()
        val edges = parts[1].trim().split(" ").filter { it.isNotEmpty() }

        edges.forEach { edge ->
            graph.addEdge(node, edge)
        }
    }

    return graph
}

// Create and visualize the graph
val input = """
    aaa: you bbb final
    bbb: ccc out
    ccc: final
    you: also_final
""".trimIndent()

val graph = parseGraphFromLines(input.lineSequence())

// Display the graph
println("Graph structure:")
println(graph)
println()

// Display as HTML table
//HTML(graph.toHtmlTable())

HTML(graph.toSvg())

// Find all paths from 'aaa' to 'final'
val allPaths = graph.findAllPaths("aaa", "final")
println("\nAll paths from 'aaa' to 'final':")
allPaths.forEachIndexed { index, path ->
    println("  Path ${index + 1}: ${path.joinToString(" -> ")}")
}

Graph structure:
aaa -> you, bbb, final
bbb -> ccc, out
ccc -> final
you -> also_final


All paths from 'aaa' to 'final':
  Path 1: aaa -> bbb -> ccc -> final
  Path 2: aaa -> final


In [3]:
// Function to read graph from file
fun readGraphFromFile(filePath: String): Graph {
    val file = File(filePath)
    if (!file.exists()) {
        throw IllegalArgumentException("File not found: $filePath")
    }
    
    return parseGraphFromLines(file.readLines().asSequence())
}

// Example of the expected file format:
println("Expected file format:")
println("wix: gix rid")
println("gix: abc def")
println("rid: xyz")
println()

// Demonstrating with the example you provided
val exampleInput = """
    wix: gix rid
    gix: abc def
    rid: xyz
""".trimIndent()

val exampleGraph = parseGraphFromLines(exampleInput.lineSequence())
println("Graph from your example:")
println(exampleGraph)
println()

HTML(exampleGraph.toSvg())

Expected file format:
wix: gix rid
gix: abc def
rid: xyz

Graph from your example:
wix -> gix, rid
gix -> abc, def
rid -> xyz



<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.0 (20251206.1807)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- wix -->
 
 wix 
 
 wix 
 
<!-- gix -->
 
 gix 
 
 gix 
 
<!-- wix->gix -->
 
 wix->gix 
 
 
 
<!-- rid -->
 
 rid 
 
 rid 
 
<!-- wix->rid -->
 
 wix->rid 
 
 
 
<!-- abc -->
 
 abc 
 
 abc 
 
<!-- gix->abc -->
 
 gix->abc 
 
 
 
<!-- def -->
 
 def 
 
 def 
 
<!-- gix->def -->
 
 gix->def 
 
 
 
<!-- xyz -->
 
 xyz 
 
 xyz 
 
<!-- rid->xyz -->
 
 rid->xyz

In [6]:
// Example usage with file reading
// Uncomment and modify the path to use with your actual file:
val graph = readGraphFromFile("/Users/jmougan/code/personal/advent/src/main/resources/year2025/day11/input.txt")

val allPaths = graph.findAllPaths("you", "out")
println("There are ${allPaths.size} paths from 'you' to 'out'")
println("The first 10 are:")
println(allPaths.take(10))

// val pathsFromSvrToOut = graph.findAllPaths("svr", "out")         // This implementation causes an java.lang.OutOfMemoryError: Java heap space
//println("There are ${pathsFromSvrToOut.size} paths from 'svr' to 'out'")
//val filteredPaths = pathsFromSvrToOut.filter { it.contains("dac") }.filter { it.contains("fft") }
//println("There are ${filteredPaths.size} from 'svr' to 'out', including 'dac' and 'fft'")


There are 500 paths from 'you' to 'out'
The first 10 are:
[[you, nns, wkx, kdt, sxy, hkq, ovs, out], [you, nns, wkx, kdt, sxy, hkq, tgd, out], [you, nns, wkx, kdt, sxy, ewj, tss, out], [you, nns, wkx, kdt, sxy, ewj, tzi, out], [you, nns, wkx, kdt, sxy, ewj, ovs, out], [you, nns, wkx, kdt, sxy, ewj, fie, out], [you, nns, wkx, kdt, sxy, uqo, tss, out], [you, nns, wkx, kdt, sxy, uqo, fie, out], [you, nns, wkx, kdt, fte, ewj, tss, out], [you, nns, wkx, kdt, fte, ewj, tzi, out]]
